In [2]:
import numpy as np

datafileName = 'DKP_Instances/toy_instance.txt'

def parseInst(filename):

    with open(filename, "r") as file:
        line = file.readline()  
        nbItems = int(line)
        
        line = file.readline()  
        capacityMax = int(line)

        profits = []
        weights = []
        
        for i in range(nbItems):
            line = file.readline()
            lineTab = line.split()
            
            profits.append(int(lineTab[1])) 
            weights.append(int(lineTab[2]))

        line = file.readline()  
        lineTab = line.split()

        nbCliques = int(lineTab[0])
        cliques = []

        for i in range(nbCliques):
            line = file.readline()
            lineTab = line.split()

            size = int(lineTab[0])
            cl = []

            for j in range(size):
                cl.append(int(lineTab[j+1]))

            cliques.append(cl)
        
    print("Nb Items : ", nbItems)
    print("Capacity : ", capacityMax)
    print("Nb cliques : ", nbCliques)
    print()
    print("Profits : ", profits)
    print("Weights : ", weights)
    print("Cliques : ", cliques)

    # For each item, the set of cliques it belongs to
    inCliques = [[] for i in range(nbItems)]

    for k in range(nbCliques):
        for i in cliques[k]:
            inCliques[i].append(k)

    print("inCliques : ", inCliques)

    return [nbItems, capacityMax, nbCliques, profits, weights, cliques, inCliques]

In [4]:
# fonction qui permet de vérifier si deux objets sont dans la même clique 

def respectClique(sol, inCliques):
    cliquesInSol = []
    for i in range(len(sol)):
        if sol[i] == 1:
            for j in inCliques[i]: 
                if j in cliquesInSol:
                    return False
                else: 
                    cliquesInSol.append(j)
    return True

# algorithme naif qui prend en entrée les données et ressort la solution optimale. 

def naif(filename): 

    # import des données de l'instance : 
    data = []
    data = parseInst(filename)
    
    #récupération des données de l'instance 
    nbItems = data[0]
    capacity = data[1]
    nbCliques = data[2]
    profits = np.array(data[3])
    poids = np.array(data[4])
    cliques = data[5]
    inCliques = data[6]

    # définition de max qui correspond à la valeur max et de solOpt qui stockera la solution optimale
    max = 0
    solOpt = np.zeros(nbItems)

    # énumération des solutions du problème de sac à dos : 
    for i in range(2**nbItems):

        sol = bin(i)[2:].zfill(nbItems) # transforme en binaire
        sol = np.frombuffer(sol.encode('utf-8'), dtype=np.uint8) - ord('0')
        
        # vérification de contrainte de capacité
        if poids.dot(sol) <= capacity:
            if(respectClique(sol, inCliques)):
                val = profits.dot(sol)
                if val > max: 
                    max = val
                    solOpt = sol
                    print(val, " ", solOpt)

    print("optimal value found : ", max, "\n", 
          "optimale solution found : ", solOpt)

naif(datafileName) 

Nb Items :  7
Capacity :  15
Nb cliques :  4

Profits :  [12, 5, 8, 10, 4, 9, 7]
Weights :  [10, 4, 6, 7, 3, 6, 5]
Cliques :  [[0, 1, 2, 3], [0, 5, 6], [1, 2, 3, 4], [3, 4, 5]]
inCliques :  [[0, 1], [0, 2], [0, 2], [0, 2, 3], [2, 3], [1, 3], [1]]
7   [0 0 0 0 0 0 1]
9   [0 0 0 0 0 1 0]
11   [0 0 0 0 1 0 1]
17   [0 0 0 1 0 0 1]
optimal value found :  17 
 optimale solution found :  [0 0 0 1 0 0 1]


In [26]:
caca = [0,0,1,1,0,1]
inCliques = [[0, 1], [0, 2], [0, 2], [0, 2, 3], [2, 3], [1, 3], [1]]

test = []

for i in range(len(caca)):
    if caca[i] == 1:
        for j in inCliques[i]:
            test.append(j)
print(test)

[0, 2, 0, 2, 3, 1, 3]


In [ ]:
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

# Données de l'instance
profits = [12, 5, 8, 10, 4, 9, 7]
weights = [10, 4, 6, 7, 3, 6, 5]
capacity = 15
cliques = [[0, 1, 2, 3], [0, 5, 6], [1, 2, 3, 4], [3, 4, 5]]

# Création du problème
prob = LpProblem("DKP", LpMaximize)

# Variables binaires
n = len(profits)
x = [LpVariable(f"x_{i}", cat="Binary") for i in range(n)]

# Fonction objectif
prob += lpSum(profits[i] * x[i] for i in range(n))

# Contraintes
prob += lpSum(weights[i] * x[i] for i in range(n)) <= capacity

for clique in cliques:
    prob += lpSum(x[i] for i in clique) <= 1

# Résolution
prob.solve()

# Affichage des résultats
print("Statut:", prob.status)
print("Profit optimal =", prob.objective.value())
print("Objets sélectionnés:")
for i in range(n):
    if x[i].value() == 1:
        print(f"Objet {i} (profit={profits[i]}, poids={weights[i]})")